## LangChain Introduction Part 2: Use Cases

**Disclaimer:**  The content in these Jupyter Notebooks is aggregated from various online sources dedicated to the documentation of the tools we will be using. Accuracy and completeness are not guaranteed. Use at your own risk and verify critical information independently.

*This Notebook is based on the [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

**Goals:**

1. Inspire you to build
2. Provide an introductory understanding of the main use cases of LangChain examples and code snippets. For an introduction to the *fundamentals* of LangChain check out [Cookbook Part 1: Fundamentals]

### **Main Use Cases**

* **Summarization** - Express the most important facts about a body of text or chat interaction
* **Question and Answering Over Documents** - Use information held within documents to answer questions or query
* **Extraction** - Pull structured data from a body of text or an user query
* **Evaluation** - Understand the quality of output from your application
* **Querying Tabular Data** - Pull data from databases or other tabular source
* **Code Understanding** - Reason about and digest code
* **Interacting with APIs** - Query APIs and interact with the outside world
* **Chatbots** - A framework to have a back and forth interaction with a user combined with memory in a chat interface
* **Agents** - Use LLMs to make decisions about what to do next. Enable these decisions with tools.


#### **Authors Note:**

* This cookbook will not cover all aspects of LangChain. It's contents have been curated to get you to building & impact as quick as possible. For more, please check out [LangChain Technical Documentation](https://python.langchain.com/en/latest/index.html)
* This notebook assumes is that you've seen part 1 of this series [Fundamentals](https://github.com/). This notebook is focused on what to do and how to apply those fundamentals.
* You'll notice I repeat import statements throughout the notebook. My intention is to lean on the side of clarity and help you see the full code block in one spot. No need to go back and forth to see when we imported a package.
* We use the default models throughout the notebook, at the time of writing they were davinci-003 and gpt-3.5-turbo. You would no doubt get better results with GPT4

Let's get started

Throughout this tutorial we will use OpenAI's various [models](https://platform.openai.com/docs/models/overview). LangChain makes it easy to [subsistute LLMs](https://langchain.com/integrations.html#:~:text=integrations%20LangChain%20provides.-,LLMs,-LLM%20Provider) so you can BYO-LLM if you want

In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
# Load environment variables
load_dotenv()
# Configure OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = os.getenv('OPENAI_API_VERSION')

### Inititate the LLMs and the embedding model needed for this notebook 

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

llm_inst = AzureOpenAI(
    openai_api_base= openai.api_base,
    openai_api_version= openai.api_version,
    deployment_name="gpt-35-turbo-instruct",
    temperature=0,
    openai_api_key= openai.api_key,
    openai_api_type = openai.api_type,
)

llm4 = AzureChatOpenAI(
    openai_api_base= openai.api_base,
    openai_api_version= openai.api_version,
    deployment_name="gpt-4-32k",
    temperature=0,
    openai_api_key= openai.api_key,
    openai_api_type = openai.api_type,
)

llm3 = AzureChatOpenAI(
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo-16k",
    temperature=0,
   openai_api_key=openai.api_key,
   openai_api_type = openai.api_type,
)

embeddings = OpenAIEmbeddings(deployment_id="text-embedding-ada-002", chunk_size=1)

In [4]:
llm_inst("Tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired."

## LangChain Use Cases

### Summarization

One of the most common use cases for LangChain and LLMs is summarization. You can summarize any piece of text, but use cases span from summarizing calls, articles, books, academic papers, legal documents, user history, a table, or financial documents. It's super helpful to have a tool which can summarize information quickly.


### Summaries Of Short Text
For summaries of short texts, the method is straightforward, in fact you don't need to do anything fancy other than simple prompting with instructions

In [5]:
from langchain.llms import OpenAI
from langchain import PromptTemplate



# Create our template
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.
%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

Let's let's find a confusing text online. *[Source](https://www.smithsonianmag.com/smart-news/long-before-trees-overtook-the-land-earth-was-covered-by-giant-mushrooms-13709647/)*

In [6]:
confusing_text = """
For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”
"""

Let's take a look at what prompt will be sent to the LLM

In [7]:
print ("------- Prompt Begin -------")

final_prompt = prompt.format(text=confusing_text)
print(final_prompt)

print ("------- Prompt End -------")

------- Prompt Begin -------

%INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.
%TEXT:

For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”


------- Prompt End -------


Finally let's pass it through the LLM

In [8]:
output = llm_inst(final_prompt)
print (output)


For a really long time, people argued about what Prototaxites was. Some said it was a type of lichen, some said it was a fungus, and others thought it was a tree. But the problem was that when scientists looked at it closely, it reminded them of different things but didn't really match with anything specific. And it was so big that when someone said it was one thing, other people got upset and said, "How could a lichen be as tall as a 20-foot tree?"


This method works fine, but for longer text, it can become a pain to manage and you'll run into token limits. Luckily LangChain has out of the box support for different methods to summarize via their [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html).

### Summaries of longer text

*Note: This method will also work for short text too*

In [9]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


Let's load up a longer document

In [10]:
with open('data/PaulGrahamEssays/good.txt', 'r') as file:
    text = file.read()

# Printing the first 285 characters as a preview
print (text[:285])

April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the
phrase that became our motto: Make something people want.  We've
learned a lot since then, but if I were choosing now that's still
the one I'd pick.


Then let's check how many tokens are in this document. [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens) is a nice method for this.

In [11]:
num_tokens = llm_inst.get_num_tokens(text)

print (f"There are {num_tokens} tokens in your file")

There are 3976 tokens in your file


While you could likely stuff this text in your prompt, let's act like it's too big and needs another method.

First we'll need to split it up. This process is called 'chunking' or 'splitting' your text into smaller pieces. I like the [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html) because it's easy to control but there are a [bunch](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) you can try

In [12]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=5000, chunk_overlap=350)
docs = text_splitter.create_documents([text])

print (f"You now have {len(docs)} docs intead of 1 piece of text")

You now have 4 docs intead of 1 piece of text


Next we need to load up a chain which will make successive calls to the LLM for us. Want to see the prompt being used in the chain below? Check out the [LangChain documentation](https://github.com/hwchase17/langchain/blob/master/langchain/chains/summarize/map_reduce_prompt.py)

For information on the difference between chain types, check out this video on [token limit workarounds](https://youtu.be/f9_BWhCI4Zo)

*Note: You could also get fancy and make the first 4 calls of the map_reduce run in parallel too*

In [13]:
# Get your chain ready to use
chain = load_summarize_chain(llm=llm_inst, chain_type='map_reduce') # verbose=True optional to see what is getting sent to the LLM

In [14]:
# Use it. This will run through the 4 documents, summarize the chunks, then get a summary of the summary.
output = chain.run(docs)
print (output)



The essay discusses the motto of Y Combinator, "Make something people want," and how it can lead to unexpected results, such as successful businesses operating like charities. Examples of successful companies are given to support this idea. The article also discusses the importance of benevolence in the success of startups, including attracting support, investors, and employees, and serving as a compass for decision-making. The author reflects on the undervaluing of being genuinely good in business and society and the importance of accountability in benevolent projects.


### Question & answering using documents as context

*[LangChain Question & Answer Docs](https://python.langchain.com/en/latest/use_cases/question_answering.html)*

In order to use LLMs for question and answer we must:

1. Pass the LLM relevant context it needs to answer a question
2. Pass it our question that we want answered

Simplified, this process looks like this "llm(your context + your question) = your answer"

* **Deep Dive** - [Question A Book](https://youtu.be/h0DHDp1FbmQ), [Ask Questions To Your Custom Files](https://youtu.be/EnT-ZTrcPrg), [Chat Your Data JS (1000 pages of Financial Reports)](https://www.youtube.com/watch?v=Ix9WIZpArm0&t=1051s), [LangChain Q&A webinar](https://www.crowdcast.io/c/rh66hcwivly0)
* **Examples** - [ChatPDF](https://www.chatpdf.com/)
* **Use Cases** - Chat your documents, ask questions to academic papers, create study guides, reference medical information

### Simple Q&A example

Here let's review the convention of `llm(your context + your question) = your answer`

In [15]:
context = """
Rachel is 30 years old
Bob is 45 years old
Kevin is 65 years old
"""

question = "Who is under 40 years old?"

Then combine them.

In [16]:
output = llm_inst(context + question)

# I strip the text to remove the leading and trailing whitespace
print (output.strip())

Rachel is under 40 years old.


As we ramp up our sophistication, we'll take advantage of this convention more.

The hard part comes in when you need to be selective about *which* data you put in your context. This field of study is called "[document retrieval](https://python.langchain.com/en/latest/modules/indexes/retrievers.html)" and tightly coupled with AI Memory.

### Using embeddings

I informally call what were about to go through as "The VectorStore Dance". It's the process of splitting your text, embedding the chunks, putting the embeddings in a DB, and then querying them. For a full video on this check out [How To Question A Book](https://www.youtube.com/watch?v=h0DHDp1FbmQ)

The goal is to select relevant chunks of our long text, but which chunks do we pull? The most popular method is to pull *similar* texts based off comparing vector embeddings.

In [17]:
# The vectorstore we'll be using
from langchain.vectorstores import FAISS
# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA
# from langchain.chains import ConversationalRetrievalChain
# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings


Let's load up a longer document

In [18]:
loader = TextLoader('data/PaulGrahamEssays/worked.txt')
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 74677 characters in that document


Now let's split our long doc into smaller pieces

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

In [22]:
# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

Now you have 29 documents that have an average of 2,931 characters (smaller pieces)


In [20]:
# Get your embeddings engine ready
# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to OpenAI
docsearch = FAISS.from_documents(docs, embeddings)

Create your retrieval engine

In [21]:
qa = RetrievalQA.from_chain_type(llm=llm_inst, chain_type="stuff", retriever=docsearch.as_retriever())

Now it's time to ask a question. The retriever will go get the similar documents and combine with your question for the LLM to reason through.

Note: It may not seem like much, but the magic here is that we didn't have to pass in our full original document.

In [22]:
query = "What does the author describe as good work?"
qa.run(query)

' The author describes working on things that are not prestigious as good work because it allows for the discovery of something real and shows that the individual has the right motives.'

If you wanted to do more you would hook this up to a cloud vector database, use a tool like metal and start managing your documents, with external data sources

### Extraction
*[LangChain Extraction Docs](https://python.langchain.com/en/latest/use_cases/extraction.html)*

Extraction is the process of parsing data from a piece of text. This is commonly used with output parsing in order to *structure* our data.

* **Deep Dive** - [Use LLMs to Extract Data From Text (Expert Level Text Extraction](https://youtu.be/xZzvwR9jdPA), [Structured Output From OpenAI (Clean Dirty Data)](https://youtu.be/KwAXfey-xQk)
* **Examples** - [OpeningAttributes](https://twitter.com/GregKamradt/status/1646500373837008897)
* **Use Cases:** Extract a structured row from a sentence to insert into a database, extract multiple rows from a long document to insert into a database, extracting parameters from a user query to make an API call

A popular library for extraction is [Kor](https://eyurtsev.github.io/kor/). We won't cover it today but I highly suggest checking it out for advanced extraction.

In [23]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# We will be using a chat model, defaults to gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI

# To parse outputs and get structured data back
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

### Vanilla Extraction

Let's start off with an easy example. Here I simply supply a prompt with instructions with the type of output I want.

In [24]:
instructions = """
You will be given a sentence with fruit names, extract those fruit names and assign an emoji to them
Return the fruit name and emojis in a python dictionary
"""

fruit_names = """
Apple, Pear, this is an kiwi
"""

In [25]:
# Make your prompt which combines the instructions w/ the fruit names
prompt = (instructions + fruit_names)

# Call the LLM
output = llm3([HumanMessage(content=prompt)])

print (output.content)
print (type(output.content))

{
  "Apple": "🍎",
  "Pear": "🍐",
  "kiwi": "🥝"
}
<class 'str'>


Let's turn this into a proper python dictionary

In [26]:
output_dict = eval(output.content)

print (output_dict)
print (type(output_dict))

{'Apple': '🍎', 'Pear': '🍐', 'kiwi': '🥝'}
<class 'dict'>


While this worked this time, it's not a long term reliable method for more advanced use cases

### Using LangChain's response schema

LangChain's response schema will does two things for us: 

1. Autogenerate a prompt with bonafide format instructions. This is great because I don't need to worry about the prompt engineering side, I'll leave that up to LangChain!

2. Read the output from the LLM and turn it into a proper python object for me

Here I define the schema I want. I'm going to pull out the song and artist that a user wants to play from a pseudo chat message.

In [27]:
# The schema I want out
response_schemas = [
    ResponseSchema(name="artist", description="The name of the musical artist"),
    ResponseSchema(name="song", description="The name of the song that the artist plays")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [28]:
# The format instructions that LangChain makes. Let's look at them
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```


In [29]:
# The prompt template that brings it all together
# Note: This is a different prompt template than before because we are using a Chat Model

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("Given a command from the user, extract the artist and song names \n \
                                                    {format_instructions}\n{user_prompt}")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

In [30]:
fruit_query = prompt.format_prompt(user_prompt="I really like So Young by Portugal. The Man")
print (fruit_query.messages[0].content)

Given a command from the user, extract the artist and song names 
                                                     The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```
I really like So Young by Portugal. The Man


In [31]:
fruit_output = llm3(fruit_query.to_messages())
output = output_parser.parse(fruit_output.content)

print (output)
print (type(output))

{'artist': 'Portugal. The Man', 'song': 'So Young'}
<class 'dict'>


Awesome, now we have a dictionary that we can use later down the line

** **Warning:** The parser looks for an output from the LLM in a specific format. Your model may not output the same format every time. Make sure to handle errors with this one. GPT4 and future iterations will be more reliable.

For more advanced parsing check out [Kor](https://eyurtsev.github.io/kor/)

### Evaluation

*[LangChain Evaluation Docs](https://python.langchain.com/en/latest/use_cases/evaluation.html)*

Evaluation is the process of doing quality checks on the output of your applications. Normal, deterministic, code has tests we can run, but judging the output of LLMs is more difficult because of the unpredictableness and variability of natural language. LangChain provides tools that aid us in this journey.

* **Examples** - [Lance Martin's Advanced](https://twitter.com/RLanceMartin) [Auto-Evaluator](https://github.com/rlancemartin/auto-evaluator)
* **Use Cases:** Run quality checks on your summarization or Question & Answer pipelines, check the output of you summarization pipeline

In [32]:
# Embeddings, store, and retrieval
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Model and doc loader
from langchain import OpenAI
from langchain.document_loaders import TextLoader

# Eval!
from langchain.evaluation.qa import QAEvalChain


In [33]:
# Our long essay from before
loader = TextLoader('data/PaulGrahamEssays/worked.txt')
doc = loader.load()

print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 74677 characters in that document


First let's do the Vectorestore dance so we can do question and answers

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

Now you have 29 documents that have an average of 2,931 characters (smaller pieces)


In [35]:
# Embeddings and docstore
docsearch = FAISS.from_documents(docs, embeddings)

Make your retrieval chain. Notice how I have an `input_key` parameter now. This tells the chain which key from a dictionary I supply has my prompt/query in it. I specify `question` to match the question in the dict below

In [36]:
chain = RetrievalQA.from_chain_type(llm=llm_inst, chain_type="stuff", retriever=docsearch.as_retriever(), input_key="question")

Now I'll pass a list of questions and ground truth answers to the LLM that I know are correct (I validated them as a human).

In [37]:
question_answers = [
    {'question' : "Which company sold the microcomputer kit that his friend built himself?", 'answer' : 'Healthkit'},
    {'question' : "What was the small city he talked about in the city that is the financial capital of USA?", 'answer' : 'Yorkville, NY'}
]

I'll use `chain.apply` to run both my questions one by one separately.

One of the cool parts is that I'll get my list of question and answers dictionaries back, but there'll be another key in the dictionary `result` which will be the output from the LLM.

Note: I specifically made my 2nd question ambigious and tough to answer in one pass so the LLM would get it incorrect

In [38]:
predictions = chain.apply(question_answers)
predictions

[{'question': 'Which company sold the microcomputer kit that his friend built himself?',
  'answer': 'Healthkit',
  'result': ' Heathkit'},
 {'question': 'What was the small city he talked about in the city that is the financial capital of USA?',
  'answer': 'Yorkville, NY',
  'result': ' The small city he talked about was Yorkville, and it was his new home in New York.'}]

We then have the LLM compare my ground truth answer (the `answer` key) with the result from the LLM (`result` key).

Or simply, we are asking the LLM to grade itself.

In [39]:
# Start your eval chain
eval_chain = QAEvalChain.from_llm(llm_inst)

# Have it grade itself. The code below helps the eval_chain know where the different parts are
graded_outputs = eval_chain.evaluate(question_answers,
                                     predictions,
                                     question_key="question",
                                     prediction_key="result",
                                     answer_key='answer')

In [40]:
graded_outputs

[{'results': ' CORRECT'}, {'results': ' CORRECT'}]

This is correct! Notice how the answer in question #1 was "Healthkit" and the prediction was "The microcomputer kit was sold by Heathkit." The LLM knew that the answer and result were the same and gave us a "correct" label. Awesome.

For #2 it knew they were not the same and gave us an "incorrect" label

## Querying Tabular Data

<p align="center">
  <img src="./pics/sql_agent.png" alt="drawing" width="600"/>
</p>

*[LangChain Querying Tabular Data Docs](https://python.langchain.com/en/latest/use_cases/tabular.html)*

The most common type of data in the world sits in tabular form (ok, ok, besides unstructured data). It is super powerful to be able to query this data with LangChain and pass it through to an LLM 

* **Deep Dive** - Coming Soon
* **Examples** - TBD
* **Use Cases:** Use LLMs to query data about users, do data analysis, get real time information from your DBs

For futher reading check out "Agents + Tabular Data" ([Pandas](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/pandas.html), [SQL](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html), [CSV](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html))

Let's query an SQLite DB with natural language. We'll look at the [San Francisco Trees](https://data.sfgov.org/City-Infrastructure/Street-Tree-List/tkzw-k3nq) dataset.

In [42]:

from langchain.sql_database import SQLDatabase 
from langchain_experimental.sql import SQLDatabaseChain

We'll start off by specifying where our data is and get the connection ready

In [43]:
sqlite_db_path = 'data/San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

Then we'll create a chain that take our LLM, and DB. I'm setting `verbose=True` so you can see what is happening underneath the hood.

In [45]:
db_chain = SQLDatabaseChain(llm=llm_inst, database=db, verbose=True)

In [46]:
db_chain.run("How many Species of trees are there in San Francisco?")



> Entering new SQLDatabaseChain chain...
How many Species of trees are there in San Francisco?
SQLQuery:SELECT COUNT(DISTINCT qSpecies) FROM SFTrees
SQLResult: [(578,)]
Answer:There are 578 species of trees in San Francisco.
> Finished chain.


'There are 578 species of trees in San Francisco.'

This is awesome! There are actually a few steps going on here.

**Steps:**
1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

Let's confirm via pandas

In [47]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

In [48]:
# Display the result in the first column first cell
print(df.iloc[0,0])

578


Nice! The answers match.

### Code Understanding

*[LangChain Code Understanding Docs](https://python.langchain.com/en/latest/use_cases/code.html)*

One of the most exciting abilities of LLMs is code undestanding. People around the world are leveling up their output in both speed & quality due to AI help. A big part of this is having a LLM that can understand code and help you with a particular task.

* **Use Cases:** Co-Pilot-esque functionality that can help answer questions from a specific library, help you generate new code

In [50]:
# Helper to read local files
import os

# Vector Support
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Model and chain
from langchain.chat_models import ChatOpenAI

# Text splitters
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader



We will do the Vectorstore dance again

I put a small python package [The Fuzz](https://github.com/seatgeek/thefuzz) in the data folder of this repo.

The loop below will go through each file in the library and load it up as a doc

In [51]:
root_dir = 'data/thefuzz'
docs = []

# Go through each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try: 
            # Load up the file as a doc and split
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

Let's look at an example of a document. It's just code!

In [52]:
print (f"You have {len(docs)} documents\n")
print ("------ Start Document ------")
print (docs[0].page_content[:300])

You have 175 documents

------ Start Document ------
# .editorconfig
# http://editorconfig.org/
root = true

[*]
charset = utf-8
end_of_line = lf
indent_size = 2
indent_style = space
insert_final_newline = true
trim_trailing_whitespace = true

[*.bat]
end_of_line = crlf

[*.go]
indent_size = 4
indent_style = tab

[*.html]
indent_size = 4

[*Makefile]



Embed and store them in a docstore. This will make an API call to OpenAI

In [ ]:
docsearch = FAISS.from_documents(docs, embeddings)

In [52]:
# Get our retriever ready
qa = RetrievalQA.from_chain_type(llm=llm_inst, chain_type="stuff", retriever=docsearch.as_retriever())

In [53]:
query = "What function do I use if I want to find the most similar item in a list of items?"
output = qa.run(query)

In [54]:
print (output)

You can use the `process.extractOne()` function from `thefuzz` package to find the most similar item in a list of items. Here's an example:

```
from thefuzz import process

choices = ["apple", "banana", "orange", "pear"]
query = "pineapple"

best_match = process.extractOne(query, choices)
print(best_match)
```

This would output `(u'apple', 36)`, which means that the most similar item to "pineapple" in the list of choices is "apple", with a similarity score of 36.


In [55]:
query = "Can you write the code to use the process.extractOne() function? Only respond with code. No other text or explanation"
output = qa.run(query)

In [56]:
print (output)

import fuzzywuzzy.process as process

choices = [
    "new york mets vs chicago cubs",
    "chicago cubs at new york mets",
    "atlanta braves vs pittsbugh pirates",
    "new york yankees vs boston red sox"
]

query = "new york mets at chicago cubs"

best = process.extractOne(query, choices)
print(best[0])


### Interacting with APIs

*[LangChain API Interaction Docs](https://python.langchain.com/en/latest/use_cases/apis.html)*

If the data or action you need is behind an API, you'll need your LLM to interact with APIs

* **Use Cases:** Understand a request from a user and carry out an action, be able to automate more real-world workflows

This topic is closely related to Agents and Plugins, though we'll look at a simple use case for this section. For more information, check out [LangChain + plugins](https://python.langchain.com/en/latest/use_cases/agents/custom_agent_with_plugin_retrieval_using_plugnplai.html) documentation.

In [18]:
from langchain.chains import APIChain

LangChain's APIChain has the ability to read API documentation and understand which endpoint it needs to call.

In this case I wrote (purposefully sloppy) API documentation to demonstrate how this works

In [19]:
api_docs = """

BASE URL: https://restcountries.com/

API Documentation:

The API endpoint /v3.1/name/{name} Used to find informatin about a country. All URL parameters are listed below:
    - name: Name of country - Ex: italy, france
    
The API endpoint /v3.1/currency/{currency} Uesd to find information about a region. All URL parameters are listed below:
    - currency: 3 letter currency. Example: USD, COP
    
Woo! This is my documentation
"""

chain_new = APIChain.from_llm_and_api_docs(llm_inst, api_docs, verbose=True)

Let's try to make an API call that is meant for the country endpoint

In [20]:
chain_new.run('Can you tell me information about france?')



> Entering new APIChain chain...
 https://restcountries.com/v3.1/name/france
[{"name":{"common":"France","official":"French Republic","nativeName":{"fra":{"official":"République française","common":"France"}}},"tld":[".fr"],"cca2":"FR","ccn3":"250","cca3":"FRA","cioc":"FRA","independent":true,"status":"officially-assigned","unMember":true,"currencies":{"EUR":{"name":"Euro","symbol":"€"}},"idd":{"root":"+3","suffixes":["3"]},"capital":["Paris"],"altSpellings":["FR","French Republic","République française"],"region":"Europe","subregion":"Western Europe","languages":{"fra":"French"},"translations":{"ara":{"official":"الجمهورية الفرنسية","common":"فرنسا"},"bre":{"official":"Republik Frañs","common":"Frañs"},"ces":{"official":"Francouzská republika","common":"Francie"},"cym":{"official":"French Republic","common":"France"},"deu":{"official":"Französische Republik","common":"Frankreich"},"est":{"official":"Prantsuse Vabariik","common":"Prantsusmaa"},"fin":{"official":"Ranskan tasavalta","c

' The response from the API provides information about France, including its official name, native name, top-level domain, currency, capital, region, languages, translations, borders, area, population, timezones, continents, flags, coat of arms, and postal code format.'

Let's try to make an API call that is meant for the currency endpoint

In [21]:
chain_new.run('Can you tell me about the currency COP?')



> Entering new APIChain chain...
 https://restcountries.com/v3.1/currency/COP
[{"name":{"common":"Colombia","official":"Republic of Colombia","nativeName":{"spa":{"official":"República de Colombia","common":"Colombia"}}},"tld":[".co"],"cca2":"CO","ccn3":"170","cca3":"COL","cioc":"COL","independent":true,"status":"officially-assigned","unMember":true,"currencies":{"COP":{"name":"Colombian peso","symbol":"$"}},"idd":{"root":"+5","suffixes":["7"]},"capital":["Bogotá"],"altSpellings":["CO","Republic of Colombia","República de Colombia"],"region":"Americas","subregion":"South America","languages":{"spa":"Spanish"},"translations":{"ara":{"official":"جمهورية كولومبيا","common":"كولومبيا"},"bre":{"official":"Republik Kolombia","common":"Kolombia"},"ces":{"official":"Kolumbijská republika","common":"Kolumbie"},"cym":{"official":"Gweriniaeth Colombia","common":"Colombia"},"deu":{"official":"Republik Kolumbien","common":"Kolumbien"},"est":{"official":"Colombia Vabariik","common":"Colombia"},"fi

' The currency COP belongs to Colombia, a country in South America with a population of over 50 million. The flag of Colombia consists of three horizontal bands of yellow, blue, and red. The capital of Colombia is Bogotá and the official language is Spanish. '

In both cases the APIChain read the instructions and understood which API call it needed to make.

Once the response returned, it was parsed and then my question was answered. Awesome 🐒

## Chatbots

*[LangChain Chatbot Docs](https://python.langchain.com/en/latest/use_cases/chatbots.html)*

Chatbots use many of the tools we've already looked at with the addition of an important topic: Memory. There are a ton of different [types of memory](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html), tinker to see which is best for you.
* **Examples** - [ChatBase](https://www.chatbase.co/?via=greg) (Affiliate link), [NexusGPT](https://twitter.com/achammah1/status/1649482899253501958?s=20), [ChatPDF](https://www.chatpdf.com/)
* **Use Cases:** Have a real time interaction with a user, provide an approachable UI for users to ask natural language questions

In [23]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

For this use case I'm going to show you how to customize the context that is given to a chatbot.

You could pass instructions on how the bot should respond, but also any additional relevant information it needs.

In [24]:
template = """
You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [25]:
llm_chain = LLMChain(
    llm= llm4, 
    prompt=prompt, 
    verbose=True, 
    memory=memory
)

In [26]:
llm_chain.predict(human_input="Is an pear a fruit or vegetable?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it


Human: Is an pear a fruit or vegetable?
Chatbot:

> Finished chain.


'Well, I\'m not sure about that, but I do know that if a pear was a vegetable, it would probably be a "pea-r"!'

In [27]:
llm_chain.predict(human_input="What was one of the fruits I first asked you about?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

Human: Is an pear a fruit or vegetable?
AI: Well, I'm not sure about that, but I do know that if a pear was a vegetable, it would probably be a "pea-r"!
Human: What was one of the fruits I first asked you about?
Chatbot:



> Finished chain.


'Oh, you\'re trying to "pea-r" into my memory, aren\'t you?'

Notice how my 1st interaction was put into the prompt of my 2nd interaction. This is the memory piece at work.

There are many ways to structure a conversation, check out the different ways on the [docs](https://python.langchain.com/en/latest/use_cases/chatbots.html)

## Agents

*[LangChain Agent Docs](https://python.langchain.com/en/latest/modules/agents.html)*

Agents are one of the hottest [🔥](https://media.tenor.com/IH7C6xNbkuoAAAAC/so-hot-right-now-trending.gif) topics in LLMs. Agents are the decision makers that can look a data, reason about what the next action should be, and execute that action for you via tools

* **Deep Dive** - [LangChain Agents Webinar](https://www.crowdcast.io/c/46erbpbz609r), much deeper dive coming soon
* **Use Cases:** Run programs autonomously without the need for human input

Examples of advanced uses of agents appear in [AutoGen](https://www.microsoft.com/en-us/research/project/autogen/) 


In [36]:
# Helpers
import os
import json

from langchain.llms import OpenAI

# Agent imports
from langchain.agents import load_tools
from langchain.agents import initialize_agent

# Tool imports
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import TextRequestsWrapper

For this example I'm going to pull google search results. You may want to do this if you need a list of websites for a research project.

You can sign up for both of these keys at the urls below

[GOOGLE_API_KEY](https://console.cloud.google.com/apis/credentials)
[GOOGLE_CSE_ID](https://programmablesearchengine.google.com/controlpanel/create)

In [37]:
# GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID', 'YourAPIKeyIfNotSet')
# GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY', 'YourAPIKeyIfNotSet')

GOOGLE_API_KEY       = "****"
GOOGLE_CSE_ID        =  "*****"



Initialize both the tools you'll be using. For this example we'll search google and also give the LLM the ability to execute python code

In [40]:
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)

requests = TextRequestsWrapper()

Put both your tools in a toolkit

In [41]:
toolkit = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    ),
    Tool(
        name = "Requests",
        func=requests.get,
        description="Useful for when you to make a request to a URL"
    ),
]

Create your agent by giving it the tools, LLM and the type of agent that it should be

In [43]:
agent = initialize_agent(toolkit, llm_inst, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

Now ask it a question, I'm going to give it one that it should go to Google for

In [ ]:
response = agent({"input":"What is the capital of canada?"})
response['output']

Great, that's correct. Now let's ask a question that requires listing the currect directory

In [73]:
response = agent({"input":"Tell me what the comments are about on this webpage https://news.ycombinator.com/item?id=34425779"})
response['output']



> Entering new AgentExecutor chain...
 I need to find out what the comments are about
Action: Search
Action Input: "comments on https://news.ycombinator.com/item?id=34425779"
Observation: About a month after we started Y Combinator we came up with the phrase that ... Action Input: "comments on https://news.ycombinator.com/item?id=34425779" .
Thought: I now know the comments are about Y Combinator
Final Answer: The comments on the webpage are about Y Combinator.

> Finished chain.


'The comments on the webpage are about Y Combinator.'

### Custom Agents with Custom Search

In [47]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

### Setup Tools

In [48]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [49]:
obj = search.run("How can I treat a sprained ankle?")
type(obj)

str

In [50]:
obj

'Avoid any activity that causes pain in the ankle. Rest your ankle, and minimize walking for the rest of the day. Ice. Apply ice to the sore part of your ankle, typically for 15 minutes on and then 15 minutes off, as much as possible for the rest of the day. Continue with 15 minutes of icing three times a day until the pain and swelling subside. Sprains happen when a ligament is torn or damaged, usually when one of your joints moves further than it should. Muscle strains happen when one of your muscles is torn. People also sometimes call strains pulled muscles or muscle tears. Providers sometimes call tendon tears strains. Sprains and strains are both common sports injuries. Acute ankle sprains are commonly seen in both primary care practices and emergency departments and can result in significant short-term morbidity, recurrent injuries, and functional instability. Appropriate initial evaluation and treatment can decrease the likelihood of these complications. This activity reviews th

In [51]:
search.run("site:webmd.com How can I treat a sprained ankle?")

'Wrap it around a bag of frozen vegetables or a plastic bag filled with ice cubes for a comfortable ice pack that can help with the swelling and pain that comes with sprains or strains. If you trip or take a wrong step, a swollen foot can be a sign of a broken bone or a sprain -- when the tough, flexible tissue that connects the bones around your ankle tears. Your foot and... All high heels boost the risk of an ankle sprain. The most common, a lateral sprain, happens when you roll onto the outside of the foot. This stretches the ankle ligaments beyond their... What to Do About Them 5 /13 Cold can help with swelling and may shrink the size of your black-and-blue mark. It also slows blood flow to the area, so less of it ends up leaking into your tissues.... Plenty of rest, proper massage, less stress, and lots of fluids (without caffeine) may help you avoid one. Migraine. Throbbing pain in the front or side of your head can put a hold on your daily ...'

In [52]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

### Prompt Template

This instructs the agent on what to do. Generally, the template should incorporate:

**tools:** which tools the agent has access and how and when to call them.

**intermediate_steps:** These are tuples of previous (AgentAction, Observation) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**input:** generic user input

In [53]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

### Set up a prompt template

In [54]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [55]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)
     

### Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc



In [56]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [57]:
output_parser = CustomOutputParser()

### Define the stop sequence 
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

### Set up the Agent
We can now combine everything to set up our agent

In [58]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm_inst, prompt=prompt)

In [59]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)
     

#### What is an Agent Executor?
Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.

In [60]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [ ]:
agent_executor.run("How can I treat a spained ankle?")

#### In Debug mode

In [ ]:
langchain.debug = True
     

agent_executor.run("How can I treat a spained ankle?")

In [ ]:
langchain.debug = False

#### Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add chat_history into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [63]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [64]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [65]:
llm_chain = LLMChain(llm=llm_inst, prompt=prompt_with_history)

In [66]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)
     

In [67]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)
     

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )
     

In [ ]:
agent_executor.run("How can I treat a sprained ankle?")


In [ ]:
agent_executor.run("How long will it take to heal?")